In [3]:
from torchtext import data
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import torch

In [4]:
path= '/Users/tancredicp/Desktop/sailing_app/data/ML Training Resources/'
all_files = glob.glob(path + "/*.csv")

In [6]:
TAGS = data.Field(unk_token = None)
TAGS.build_vocab(['0','S', 'C','G','T','B','D', 'A'])

Data = []
for filename in all_files:
    temporary_dict = {}

    dfSpecific = pd.read_csv(filename, index_col=None, header=0)
    dfSpecific.fillna(0, inplace= True)
    
    dfSpecificY = dfSpecific['tag']
    temporary_dict['tags'] = TAGS.numericalize((np.array(dfSpecificY)))
    

    dfSpecificX = dfSpecific.drop(columns=['tag', 'tag_name', 'boat_class', 'time'])
    
    scaler = StandardScaler()
    columnsBeingStandardize = ['lat', 'lon', 'sog', 'heel', 'clew_load','pitch']
    dfSpecificX[columnsBeingStandardize] = scaler.fit_transform(dfSpecificX[columnsBeingStandardize])
    
    
    array1 = np.array(dfSpecificX[['lat', 'lon', 'sog', 'cog']])
    
    temp = torch.from_numpy(array1)
    
    temporary_dict['track'] = temp
    
    Data.append(temporary_dict)
    del(temporary_dict)
    break

In [18]:
Data[0]['tags'].shape, Data[0]['track'].shape

(torch.Size([1, 3594]), torch.Size([3594, 4]))

In [ ]:
#create the moving window data:

In [19]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm1 = nn.LSTM(input_size = 4, hidden_size = 32, num_layers = 2, dropout = 0.5)
        self.fc1 = nn.Linear(32, 120)
        self.fc2 = nn.Linear(120,9)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(0)
        x = x.unsqueeze(0)
        x, _ = self.lstm1(x)
#         print(x.shape)
        x = F.dropout(x, p = 0.9)
#         print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.softmax(x)
        return x

